In [6]:
import sys
sys.path.append("..")
import authenticate
import geopandas as gpd
import geemap
import pandas as pd
import ee

# Authenticate Earth Engine
authenticate.initialize_earth_engine()

In [7]:
# Função para calcular o SAVI
def calculate_savi(image):
    savi = image.expression(
        '((1 + L) * (NIR - RED)) / (NIR + RED + L)',
        {
            'NIR': image.select('B8'),  # Banda NIR (Near Infrared)
            'RED': image.select('B4'),  # Banda RED
            'L': 0.5, # Fator de correção do solo
        }).rename('SAVI')
    return image.addBands(savi)

# Função para aplicar a máscara SCL para remover nuvens e sombras
def apply_scl_mask(image):
    scl = image.select('SCL')
    # Define valores que representam vegetação
    vegetation_mask = scl.eq(4)  # 4 representa vegetação, você pode ajustar conforme necessário
    return image.updateMask(vegetation_mask)

In [8]:
gdf = gpd.read_file('..\data\parque_serra_cipo.gpkg')
gdf_ee = geemap.geopandas_to_ee(gdf)

In [9]:
# Define as datas de início e término
start_date = '2022-11-01'
end_date = '2022-12-31'

In [10]:
# Filtra a coleção Sentinel-2
collection = (ee.ImageCollection('COPERNICUS/S2_SR_HARMONIZED')
              .filterBounds(gdf_ee.geometry())  # Use a geometria do GeoDataFrame
              .filterDate(ee.Date(start_date), ee.Date(end_date)))

In [11]:
# Mapeia a função de cálculo do savi sobre a coleção
collection_with_savi = collection.map(calculate_savi)

# Aplica a máscara SCL à coleção em uma etapa separada
collection_with_mask = collection_with_savi.map(apply_scl_mask)

# Converte a coleção em uma lista
image_list = collection_with_mask.toList(collection_with_mask.size())

# Inicializa uma lista para armazenar os dados
all_data = []

# Loop sobre as imagens na lista
for i in range(collection_with_mask.size().getInfo()):
    image = ee.Image(image_list.get(i))
    
    # Calcula o SAVI médio na região de interesse
    mean_savi_region = image.reduceRegion(
        reducer=ee.Reducer.mean(),
        geometry=gdf_ee.geometry(),  # Use a geometria do GeoDataFrame
        scale=10,
        maxPixels=1e13
    )
    
    date = image.date().format("yyyy-MM-dd").getInfo()
    savi_mean = mean_savi_region.get('NDVI').getInfo()
    
    data = {'date': date, 'savi_mean': savi_mean}
    all_data.append(data)

EEException: Dictionary.get: Dictionary does not contain key: NDVI.

In [ ]:
# Cria um DataFrame a partir dos dados
df = pd.DataFrame(all_data)

In [ ]:
df

,date,ndvi_mean
0,2022-11-02,NaN
1,2022-11-04,0.531155
2,2022-11-07,0.532685
3,2022-11-09,0.608312
4,2022-11-12,NaN
5,2022-11-14,0.531042
6,2022-11-17,0.546167
7,2022-11-19,0.533477
8,2022-11-22,NaN
9,2022-11-24,NaN
